# University Schedule
## Objective 
This notebook has the objective to explore the problem and explain every step to resolve it. After of development of this, I'll implement it in a .py archive for simple execution.
## The problem
The most university in Ecuador has a system to matriculate students which consist of the next Logic:
- Each class has multiple options in days and hours as well as different professors
- Each class has prerequisites for taking it, if a student doesn't approve these requirements can't matriculate in it.
- Each class has a maximum number of students.
- At the start of all semesters or cycles, each student has multiple options for their classes and it is their work to match all classes to create their schedule.
- Creating a schedule isn't all work because if the class is complete and the student couldn't matriculate in it, this schedule may be useless and It's necessary to use another.
The search for the best schedule and some options is a tiring task and It may take several hours and generate much frustration.
## The propuse solution
The solution developed use the options of each class saved in .xls